# torch._dynamo.export 结论（2023-12-07）
1. 支持静态 flag 控制
2. 不支持 Tensor 控制，torch 建议使用 `functorch.experimental.control_flow.cond` 无法调通，同样报错不支持

In [1]:
import torch

In [2]:
from torch import nn

In [28]:
def fn_flag(x):
    if x.shape[0] > 2:
        return torch.relu(x)
    return torch.sigmoid(x)


x = torch.randn(2)
out = torch._dynamo.export(fn_flag)(x)
out.graph_module.print_readable()

class GraphModule(torch.nn.Module):
    def forward(self, x):
        arg0: f32[s0], = fx_pytree.tree_flatten_spec(([x], {}), self._in_spec)
        # File: /tmp/ipykernel_174419/1864720726.py:2, code: if x.shape[0] > 2:
        size = arg0.size()
        getitem = size[0];  size = None
        gt = getitem > 2;  getitem = None
        
        # File: /tmp/ipykernel_174419/1864720726.py:4, code: return torch.sigmoid(x)
        sigmoid = torch.sigmoid(arg0);  arg0 = None
        return pytree.tree_unflatten([sigmoid], self._out_spec)
        


'class GraphModule(torch.nn.Module):\n    def forward(self, x):\n        arg0: f32[s0], = fx_pytree.tree_flatten_spec(([x], {}), self._in_spec)\n        # File: /tmp/ipykernel_174419/1864720726.py:2, code: if x.shape[0] > 2:\n        size = arg0.size()\n        getitem = size[0];  size = None\n        gt = getitem > 2;  getitem = None\n        \n        # File: /tmp/ipykernel_174419/1864720726.py:4, code: return torch.sigmoid(x)\n        sigmoid = torch.sigmoid(arg0);  arg0 = None\n        return pytree.tree_unflatten([sigmoid], self._out_spec)\n        '

In [31]:
# 不支持 Tensor 作为判断条件，会报错
# 其它 python 类型数据不会报错
flag = (torch.randn(1)[0] > 0.5).item()

# import numpy as np
# flag = np.random.uniform(size=(1,))[0] > 0.5

def fn_flag(x):
    if flag:
        return torch.relu(x)
    return torch.sigmoid(x)


x = torch.randn(2)
out = torch._dynamo.export(fn_flag)(x)
out.graph_module.print_readable()

class GraphModule(torch.nn.Module):
    def forward(self, x):
        arg0: f32[s0], = fx_pytree.tree_flatten_spec(([x], {}), self._in_spec)
        # File: /tmp/ipykernel_174419/764942184.py:9, code: return torch.sigmoid(x)
        sigmoid = torch.sigmoid(arg0);  arg0 = None
        return pytree.tree_unflatten([sigmoid], self._out_spec)
        


'class GraphModule(torch.nn.Module):\n    def forward(self, x):\n        arg0: f32[s0], = fx_pytree.tree_flatten_spec(([x], {}), self._in_spec)\n        # File: /tmp/ipykernel_174419/764942184.py:9, code: return torch.sigmoid(x)\n        sigmoid = torch.sigmoid(arg0);  arg0 = None\n        return pytree.tree_unflatten([sigmoid], self._out_spec)\n        '

In [3]:
def fn_data_dependent_control_flow(x):
    if x.sum() > 0:
        return torch.relu(x)
    else:
        return torch.sigmoid(x)

In [7]:
def test_fn_data_dependent_control_flow():
    a = torch.randn(10)
    out = torch._dynamo.export(fn_data_dependent_control_flow)(a)
    guards = out.guards
    gm = out.graph_module
    print(gm)
    for guard in guards:
        print(guard)


test_fn_data_dependent_control_flow()

UserError: Dynamic control flow is not supported at the moment. Please use functorch.experimental.control_flow.cond to explicitly capture the control flow

from user code:
   File "/tmp/ipykernel_174419/1401405424.py", line 2, in fn_data_dependent_control_flow
    if x.sum() > 0:

Set TORCH_LOGS="+dynamo" and TORCHDYNAMO_VERBOSE=1 for more information


In [25]:
# 尝试过 lambda function，x.sum(), x.shape 等判断条件，都不支持
def true_fn(x):
    return torch.relu(x)


def false_fn(x):
    return torch.sigmoid(x)


def fn_data_dependent_control_flow(x):
    return torch._higher_order_ops.cond(x.shape[0] > 0, true_fn, false_fn, (x,))

In [24]:
def test_fn_data_dependent_control_flow():
    a = torch.randn(10)
    out = torch._dynamo.export(fn_data_dependent_control_flow)(a)
    guards = out.guards
    gm = out.graph_module
    print(gm)
    for guard in guards:
        print(guard)


test_fn_data_dependent_control_flow()

Unsupported: call_function PythonModuleVariable() [SymNodeVariable(), UserFunctionVariable(), UserFunctionVariable(), TupleVariable()] {}

from user code:
   File "/tmp/ipykernel_174419/2026860618.py", line 6, in fn_data_dependent_control_flow
    return torch._higher_order_ops.cond(x.shape[0]>0, true_fn, false_fn, (x, ))

Set TORCH_LOGS="+dynamo" and TORCHDYNAMO_VERBOSE=1 for more information
